# ДЗ

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

## Практика:

4) Поэкспериментировать с ALS (grid-search) ********
   * Подобрать лучшие параметры
   * Написать отчет об увиденном

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [4]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [6]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [7]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."
2,5,"[913077, 1118028, 1386668]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108..."
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10..."


In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [10]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 
sparse_user_item = csr_matrix(user_item_matrix)
user_item_matrix.head()

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
def fit_model(factors=64, regularization=0.05, u_i_matrix='original'):

    all_variants_matrix = {
        'original' : user_item_matrix,
        'tfidf': tfidf_weight(user_item_matrix.T).T,
        'bm25': bm25_weight(user_item_matrix.T).T
    }

    model = AlternatingLeastSquares(factors=factors,
                                    regularization=regularization,
                                    iterations=15,
                                    calculate_training_loss=True,
                                    num_threads=4,
                                    use_gpu=False)

    model.fit(csr_matrix(all_variants_matrix[u_i_matrix]).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    return model

def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

def grid_search(factors=None, regularizations=None, user_item_matrixes=None):
    global result
    if factors is None:
        factors = [64]
    if regularizations is None:
        regularizations = [0.05]
    if user_item_matrixes is None:
        user_item_matrixes = ['original']
    scores = pd.DataFrame()
    for factor in factors:
        for regularization in regularizations:
            for matrix in user_item_matrixes:

                name_test = f'matrix: {matrix}, factor: {factor}, regularization: {regularization}'

                model = fit_model(factors=factor, regularization=regularization, u_i_matrix=matrix)

                result[name_test] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

                score = result.apply(lambda row: precision_at_k(row[name_test], row['actual']), axis=1).mean()
                scores = scores.append(pd.DataFrame([score], columns=['score'], index=[name_test]))
                print('score:', round(score,4), name_test)

    return result, scores.sort_values(by='score', ascending=False)

In [13]:
%%time

result, scores = grid_search(factors=[32, 64, 128],
                             regularizations=[0.01, 0.03, 0.05],
                             user_item_matrixes=['original', 'tfidf', 'bm25'])

  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1632 matrix: original, factor: 32, regularization: 0.01


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1687 matrix: tfidf, factor: 32, regularization: 0.01


  0%|          | 0/15 [00:00<?, ?it/s]

...

score: 0.1491 matrix: original, factor: 128, regularization: 0.05


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1506 matrix: tfidf, factor: 128, regularization: 0.05


  0%|          | 0/15 [00:00<?, ?it/s]

score: 0.1935 matrix: bm25, factor: 128, regularization: 0.05
CPU times: user 1h 6s, sys: 13min 38s, total: 1h 13min 44s
Wall time: 13min 2s


In [14]:
scores

,score
"matrix: bm25, factor: 128, regularization: 0.05",0.193471
"matrix: bm25, factor: 128, regularization: 0.01",0.191763
"matrix: bm25, factor: 128, regularization: 0.03",0.191462
"matrix: bm25, factor: 64, regularization: 0.03",0.176595
"matrix: bm25, factor: 64, regularization: 0.05",0.176092
"matrix: bm25, factor: 64, regularization: 0.01",0.173079
"matrix: tfidf, factor: 32, regularization: 0.01",0.168659
"matrix: tfidf, factor: 32, regularization: 0.03",0.166951
"matrix: tfidf, factor: 32, regularization: 0.05",0.166148
"matrix: original, factor: 32, regularization: 0.01",0.163235


Наиболее оптимальные параметры - matrix: bm25, factor: 128, regularization: 0.05